# Auxiliar 00 - Bibliotecas y tipos de Python para fechas y horas

## Biblioteca estándard de python

Almacenamos con formato `datetime` a partir de una cadena con formato empleando el método `strptime`. Luego, `strftime` nos devuelve al formato de cadena de caracteres.

In [ ]:
from datetime import date, time, datetime, timedelta

dt0 = datetime.strptime('2026-01-01 00:00:00', '%Y-%m-%d %H:%M:%S')
dt1 = datetime.strptime('2026-01-02', '%Y-%m-%d')
print('Confirmamos el tipo de dato')
print(f'Tipo: {type(dt1)}\n')
print('Confirmamos que, por defecto, configura horas, minutos y segundos en cero')
print(f'dt1: {dt1}\n') 
print(f'Modificamos el estilo de impresión para dt0: {dt0.strftime('%Y%m%dT%H%M%S')}')

### Operaciones básicas con fechas y horas

Los objetos `timedelta` representan una duración, o la diferencia entre dos instancias de tipo `timedate`o `date`.


In [ ]:
interval0 = dt1 - dt0
print(f'Intervalo entre dt1 y dt0 es: {interval0}')
print(f'Interval es del tipo {type(interval0)}')
print(f'Extraemos el intervalo en días: {interval0.days}')

# Definimos un intervalo

interval1 = timedelta(days=2)
print(f'dto + 2 días: {dt0 + interval1}')


Muchas veces necesitamos convertir el dato `timedelta` en `float` para realizar cálculos. Podemos usar el método `.total_seconds()` o dividir por un intervalo de duración conocida.

In [ ]:
duration = interval0 / timedelta(hours=1)
print(f'Longitud del intervalo en horas {duration}')
print(f'Tipo de dato para duration: {type(duration)}')

## Biblioteca Numpy

El tipo de dato se llama `datetime64` porque `datetime` ya había sido tomado por la biblioteca estándard de Python.

La forma más rápida de cargar un dato es en formato ISO 'YYYY-MM-DD HH:MM:SS', eventualmente, incluyendo solo la fecha.

Es interesante porder generar un arreglo de fechas con `np.arange()`.


In [ ]:
import numpy as np
dates = np.arange(np.datetime64('2024-01-01'), np.datetime64('2024-01-07'), np.timedelta64(1, 'D'))
dates

In [ ]:
dates = np.arange(np.datetime64('2024-01-01'), np.datetime64('2024-01-07'), np.timedelta64(24*60*60, 's'))
dates

Vemos que los datos se pueden almacenar con un tipo diferente según si guardan información de días, horas, minutos, segundos, etc.

    "Datetimes are always stored with an epoch of 1970-01-01T00:00...The length of the span is the range of a 64-bit integer times the length of the date or unit."


Normalmente, se convierte a cadena de caracteres con el método `.astype('str')`. Para customizar la salida se debe pasar a `datetime` de la biblioteca estandard con el método `.item()`.

### Operaciones básicas con fechas de numpy

Generamos una lista con datos cada seis horas y buscamos convertirlo a días desde le comienzo de la lista.


In [ ]:

dates = np.arange(np.datetime64('2024-01-01'), np.datetime64('2024-02-01'),np.timedelta64(6, 'h'))
dt = dates - dates[0]
#dt = dt.astype('timedelta64[D]')
dt

In [ ]:
# Aprovechamos que ya está la diferencia en horas
dt.astype('float32')/24

## Biblioteca Pandas

Pandas puede usar los tipos de datos tanto de la biblioteca estandard como de Numpy. `pd.to_datetime()` es el mecanismo típico para cargar los datos.


In [ ]:
import pandas as pd

dti = pd.to_datetime(dates)
dti

### Operaciones básicas con fechas en Pandas


Generamos la lista con el método `DatetimeIndex()` cada seis horas y queremos pasarlo a días desde le comienzo de la lista. Es importante para operaciones de remuestroe que `DatetimeIndex()` sea el índice del `DataFrame`.


In [ ]:
x = pd.date_range(start='2014-01-01', end='2014-02-01', freq='6h')
df = pd.DataFrame(index=x, columns=['time since'])
df.head()

In [ ]:
df.index[0]

In [ ]:
df['time since'] = (df.index - df.index[0]).days
# Eventualmente completar con
df['time since'] = np.float32(df['time since'].values)


# El método `.timestamp()` nos da la fecha en segundos
df['time since'] = (df.index.map(pd.Timestamp.timestamp) - df.index[0].timestamp()) / (24*60*60)
# dividimos finalmente por 1 día

# Opción con más control
#basedate = df.index[0]
#df['time since'] = df.apply(lambda x: (x.name - basedate).to_numpy().astype('timedelta64[h]').astype('float32')/24, axis=1)




In [ ]:
df.head()


Con `.as_freq()` podemos introducir valores intermedios. Además, `.resample()` permite agrupar y operar para obtener un valor único correspondiente a la nueva frecuencia.

In [ ]:
df.asfreq(freq='3h')
df.resample('24h').sum()

Asimismo, encontramos utilidades como `.shift()` o . `.rolling()` que  reproduce ventanas móviles a las que podemos aplicar difrentes operaciones como suma 

In [ ]:
df.shift(-1)
df.rolling('24h').sum()